### Covid Cases By State

https://www.cdc.gov/covid-data-tracker/#cases

How can we find states that have similar numbers of COVID cases (total or per capita)? 
For example, let's write a SQL query to find all states that have similar infection numbers. 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/US_MAP_DATA.csv')

In [3]:
df

,abbr,fips,jurisdiction,Total Cases,Total Death,Death_100k,CasesInLast7Days,RatePer100000
0,AK,2,Alaska,2729,22,3.0,688,370
1,AL,1,Alabama,82530,1493,31.0,12104,1688
2,AR,5,Arkansas,40181,428,14.0,5516,1333
3,AS,60,American Samoa,0,0,NaN,0,0
4,AZ,4,Arizona,165934,3408,48.0,17251,2314
5,CA,6,California,466550,8518,22.0,65781,1179
6,CO,8,Colorado,45314,1807,32.0,4255,796
7,CT,9,Connecticut,49077,4423,124.0,981,1374
8,DC,11,District of Columbia,11945,583,83.0,518,1700
9,DE,10,Delaware,14602,581,60.0,810,1510


In [4]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

You may notice that "Total Cases" has a space in it. Spaces aren't allowed in SQL when referring to table names. A quick fix here is to put back ticks aroudn it, `Total Cases`. Note that these are ticks, not quotes! (you'll generally find them in the upper left corner of your keyboard). 

In [5]:
pysqldf("""
SELECT 
    abbr, `Total Cases`
FROM 
    df
""")

,abbr,Total Cases
0,AK,2729
1,AL,82530
2,AR,40181
3,AS,0
4,AZ,165934
5,CA,466550
6,CO,45314
7,CT,49077
8,DC,11945
9,DE,14602


If you'd rather not deal with this repeatedly, you can rename the column in SQL or pandas - we'll use pandas here.

In [6]:
df.rename(columns={'Total Cases': 'Total_Cases'}, inplace=True)

Let's start by finding states that have total cases within 10000 of each other. 

In [7]:
within_10000 = pysqldf("""
SELECT
    a.abbr as abbr_a, 
    b.abbr as abbr_b, 
    a.Total_Cases as total_a,
    b.Total_Cases as total_b
FROM   
    df a
JOIN 
    df b 
ON
    ABS(total_a - total_b) < 10000
AND
    abbr_a <> abbr_b
AND
    abbr_a > abbr_b
""")

In [8]:
within_10000

,abbr_a,abbr_b,total_a,total_b
0,AS,AK,0,2729
1,CO,AR,45314,40181
2,CT,AR,49077,40181
3,CT,CO,49077,45314
4,DC,AK,11945,2729
5,DE,DC,14602,11945
6,FSM,AK,0,2729
7,FSM,AS,0,0
8,GA,AZ,175052,165934
9,GU,AK,351,2729


#### Using a Cross Product

This question came up during the most recent SQL workshop - what happens when you don't specify a JOIN condition. 
Are there times when you'd want the entire cross-product of two tables?

Yes, there are times when you want to take the entire cross product of two tables, to get every possible pairing of rows from two (or more) different tables. Keep in mind that this can produce extremely large result sets quickly! If you're dealing with large tables (or multiple tables), you may want to reconsider your query and try to find a way reduce the result set. 

We can either use a JOIN with no ON condition or use a CROSS JOIN. I prefer to use the CROSS JOIN syntax, as it makes it clear in the query that I really do intend to take the full cross product, and haven't just forgotten to include an ON condition in my SQL query. Forgetting to include an ON clause in a JOIN is a fairly common SQL bug (actually, when my queries are taking an unexpectedly long time to run, this is the first thing I check). 

To illustrate, let's write a SQL query to find each state and state with the closest number of cases.

We'll start by taking the cross product of all states, and print out the difference between the case count for the two states

In [34]:
all_states_diff = pysqldf("""
SELECT
    a.abbr AS abbr_a,
    b.abbr AS abbr_b,
    ABS(a.total_cases - b.total_cases) diff
FROM
    df a
CROSS JOIN
    df b
WHERE
   abbr_a <> abbr_b
AND
    a.total_cases <> 0
AND 
    b.total_cases <> 0
""")

In [35]:
all_states_diff

,abbr_a,abbr_b,diff
0,AK,AL,79801
1,AK,AR,37452
2,AK,AZ,163205
3,AK,CA,463821
4,AK,CO,42585
5,AK,CT,46348
6,AK,DC,9216
7,AK,DE,11873
8,AK,FL,434138
9,AK,GA,172323


Next, we can write a query to find the state with the smallest difference for each state

There are a few ways to do this. One approach is to use MIN and GROUP BY. However, keep in mind the query below is allowable only in a few versions of SQLite. Generally, all terms in an aggregation query must be either in the aggregation function or in the GROUP BY clause. In the query below, abbr_b is in neither. 

This syntax is permitted and will work only with MIN and MAX aggregations, largely because MIN and MAX can be matched with a unique row, whereas SUM or AVG cannot. 

For more info, see 

In [36]:
pysqldf("""
SELECT 
    abbr_a, abbr_b, MIN(diff)
FROM
    all_states_diff
GROUP BY 
    abbr_a
""")

,abbr_a,abbr_b,MIN(diff)
0,AK,WY,140
1,AL,SC,1579
2,AR,UT,844
3,AZ,IL,9039
4,CA,FL,29683
5,CO,NV,163
6,CT,CO,3763
7,DC,DE,2657
8,DE,PR,1459
9,FL,CA,29683


For a more generalizable query, you can use a LEFT JOIN. 

Notice that there is a difference between the this result set and the GROUP by approach above. This set will list rows with ties (ie., when the difference between a state and two or more other states is equal).

In [37]:
pysqldf("""
SELECT 
    a.abbr_a, a.abbr_b, a.diff
FROM 
    all_states_diff a                   
LEFT JOIN 
    all_states_diff b            
ON 
    A.abbr_b = b.abbr_b 
    AND 
    A.diff > b.diff
WHERE 
    b.diff is NULL 
""")

,abbr_a,abbr_b,diff
0,AK,WY,140
1,AL,SC,1579
2,AR,UT,844
3,CA,FL,29683
4,CA,US,3873447
5,CO,CT,3763
6,CO,NV,163
7,DE,DC,2657
8,FL,CA,29683
9,FL,TX,42602


### Exercise - Non-symmetric pairs

Some of these pairs repeat - for example, AK's closest match is WY at 140, and WY's closest match is AK. 

Which pairs are only work in one direction? For example, Texas's closest match is Florida, but Florida's closest match is California.
